In [1]:
import os
# import unstructured
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Install poppler & tesseract and set path 

In [2]:
import os
os.environ["PATH"] = os.getenv('PATH') + ':' + '/opt/homebrew/Cellar/tesseract/5.3.4_1/bin' + ':' + '/opt/homebrew/Cellar/poppler/24.04.0/bin'
print(os.getenv('PATH'))

/Library/Developer/CommandLineTools/usr/bin:/Users/dileepbarla/Library/Python/3.9/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/poppler/24.04.0/bin


In [3]:
file_name = "pdf_files/SW_school_report.pdf"

# Extracts the elements from the PDF
elements = partition_pdf(
    filename=file_name,

    # Unstructured Helpers
    strategy="hi_res", 
    infer_table_structure=True, 
    model_name="yolox"
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
print(elements)

[<unstructured.documents.elements.Image object at 0x378751130>, <unstructured.documents.elements.Title object at 0x34facb3a0>, <unstructured.documents.elements.NarrativeText object at 0x378839250>, <unstructured.documents.elements.Title object at 0x378839b50>, <unstructured.documents.elements.Title object at 0x378839100>, <unstructured.documents.elements.Title object at 0x378751280>, <unstructured.documents.elements.Table object at 0x378839e50>, <unstructured.documents.elements.Text object at 0x37843c820>, <unstructured.documents.elements.NarrativeText object at 0x378839700>, <unstructured.documents.elements.NarrativeText object at 0x37843cd00>, <unstructured.documents.elements.Table object at 0x3780c7100>, <unstructured.documents.elements.Text object at 0x37843c5e0>, <unstructured.documents.elements.Text object at 0x37843cf40>, <unstructured.documents.elements.Text object at 0x37843cd30>, <unstructured.documents.elements.Text object at 0x37843c0a0>, <unstructured.documents.elements.Ti

In [5]:
elements_to_json(elements, "SW_school_report.json")

Retrieve tables from json

In [7]:
import json
def process_json_file(input_filename):
    
# Read the JSON file
    with open(input_filename, 'r') as file:
        data = json.load(file)

# Iterate over the JSON data and extract required table elements
    extracted_elements = []
    for entry in data:
        if entry["type"] == "Table":
            extracted_elements.append(entry["metadata"]["text_as_html"])

# Write the extracted elements to the output file
    with open("SW_school_report_tbl.txt", 'w') as output_file:
        for element in extracted_elements:
            output_file.write(element + "\n\n") # Adding two newlines for separation    

In [8]:
process_json_file("SW_school_report.json")

In [9]:
from langchain.document_loaders import TextLoader
loader = TextLoader("SW_school_report_tbl.txt")
documents = loader.load()

In [10]:
print(documents)

[Document(page_content='<table><tr><td>Grade Range</td><td>9-12</td></tr><tr><td>Enrollment</td><td>1,325</td></tr><tr><td colspan="2">Per Pupil Expenditures’ $17,134</td></tr><tr><td>Total Expenditures’</td><td>$21,965,866</td></tr></table>\n\n<table><tr><td colspan="2">Contents</td></tr><tr><td colspan="2">Students.</td></tr><tr><td>EGUCatOrSmescssssesresersesrrersrssrertesrrersesersstrsrrsesereressesrssesestts</td><td>2</td></tr><tr><td colspan="2">Instruction ANd RESOUMCES...........ssscssessssesesssscsessseceessees 2</td></tr><tr><td>Performance and Accountability.</td><td></td></tr></table>\n\n<table><thead><th></th><th colspan="2">School</th><th rowspan="2">District Percent of Total (%)</th></thead><thead><th></th><th>Count</th><th>Percent of Total (%)</th><th></th></thead><tr><td>Female</td><td>*</td><td>*</td><td>*</td></tr><tr><td>Male</td><td>666</td><td>50.1</td><td>51.2</td></tr><tr><td>Non-Binary</td><td>*</td><td>*</td><td>*</td></tr><tr><td>American Indian or Alaska Nat

In [11]:
from langchain.text_splitter import CharacterTextSplitter
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1157, which is longer than the specified 1000
Created a chunk of size 1035, which is longer than the specified 1000
Created a chunk of size 1929, which is longer than the specified 1000
Created a chunk of size 3232, which is longer than the specified 1000


In [12]:
from dotenv import load_dotenv
load_dotenv()
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
import chromadb
from langchain_chroma import Chroma


db = Chroma.from_documents(docs, embeddings)


In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Initialize your model and retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
# List of questions
questions = [
"How many students enrolled in October 1, 2022 Enrollment?",
"How many Asian students enrolled in October 1, 2022 Enrollment?",
"How many female students have Chronic Absenteeism and Suspension/Expulsion?",
"What's Shareholder's net income?"
]

In [18]:
# Store responses in output_list
output_list = []

for query in questions:
    response = qa_chain({"query": query})
    output_list.append(response)

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [19]:

import pprint

# Use pprint to pretty print the output list
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output_list)

[   {   'query': 'How many students enrolled in October 1, 2022 Enrollment?',
        'result': 'The number of students enrolled on October 1, 2022, is '
                  '1,325.'},
    {   'query': 'How many Asian students enrolled in October 1, 2022 '
                 'Enrollment?',
        'result': 'Based on the provided data, there were 335 Asian students '
                  'enrolled in the October 1, 2022 Enrollment.'},
    {   'query': 'How many female students have Chronic Absenteeism and '
                 'Suspension/Expulsion?',
        'result': 'In the provided data, there are 80 female students with '
                  'Chronic Absenteeism and 13 female students with '
                  'Suspension/Expulsion.'},
    {   'query': "What's Shareholder's net income?",
        'result': "I don't have the specific information needed to calculate "
                  "the Shareholder's net income based on the provided "
                  'context.'}]
